In [107]:
!pip install torchsummary

In [108]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,random_split
from torchvision import datasets, transforms, models 
from torchvision.utils import make_grid
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
import torch.optim as optim
from tqdm import tqdm
from torchsummary import summary

# Data Visualization
import plotly.express as px
import matplotlib.pyplot as plt

import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.manifold import TSNE
import shutil

import warnings
warnings.filterwarnings("ignore")

In [109]:
BATCH_SIZE = 128
LR = 0.0001

In [110]:
from torchvision import transforms

# Define the preprocessing transforms
transform = transforms.Compose([
    transforms.RandomRotation(10),      # rotate +/- 10 degrees
    transforms.RandomHorizontalFlip(),  # reverse 50% of images
    transforms.Resize(224),             # resize shortest side to 224 pixels
    transforms.CenterCrop(224),         # crop longest side to 224 pixels at center
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225])
])

dataset=datasets.ImageFolder(root='/kaggle/input/flowers/flowers',transform=transform)

In [111]:
train_size = int(0.7 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

In [112]:
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [113]:
class_names=dataset.classes
print(class_names)
print(len(class_names))

['astilbe', 'bellflower', 'black_eyed_susan', 'calendula', 'california_poppy', 'carnation', 'common_daisy', 'coreopsis', 'daffodil', 'dandelion', 'iris', 'magnolia', 'rose', 'sunflower', 'tulip', 'water_lily']
16


In [114]:
class ResNet18(nn.Module):
    def __init__(self, num_classes=5):
        super(ResNet18, self).__init__()
        self.model = models.resnet18(pretrained=False)
        self.model.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.model(x)
        return x

In [115]:
class VGG16(nn.Module):
    def __init__(self, num_classes=5):
        super(VGG16, self).__init__()
        self.features = models.vgg16(pretrained=False).features
        self.classifier = nn.Linear(512 * 7 * 7, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [116]:
class MobileNetV2(nn.Module):
    def __init__(self, num_classes=5):
        super(MobileNetV2, self).__init__()
        self.features = models.mobilenet_v2(pretrained=False).features
        self.classifier = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = nn.functional.adaptive_avg_pool2d(x, 1).reshape(x.shape[0], -1)
        x = self.classifier(x)
        return x

In [117]:
my_gpu = torch.cuda.is_available()
print(f'Train on gpu: {my_gpu}')
device = torch.device('cuda' if my_gpu else 'cpu')

Train on gpu: True


In [118]:
model1 = ResNet18(num_classes = len(class_names)).to(device)
model2 = VGG16(num_classes = len(class_names)).to(device)
model3 = MobileNetV2(num_classes = len(class_names)).to(device)

In [119]:
fetch_model_Resnet18_D3 = torch.load('/kaggle/input/model-weigts/Resnet18_dataset3.pth')
fetch_model_vgg16_D3 = torch.load('/kaggle/input/d3-vgg16-model/d3_vgg16_model_checkpoint.pth')
fetch_model_MV2_D3 = torch.load('/kaggle/input/model-weigts/d3_mnv2_model_checkpoint.pth')

In [120]:
print(f"RESNET18 -> Model1 Keys")
print(fetch_model_Resnet18_D3.keys())
model1.load_state_dict(fetch_model_Resnet18_D3['state_dict'])
model1.eval()

RESNET18 -> Model1 Keys
dict_keys(['epoch', 'state_dict', 'optimizer', 'train_loss', 'val_loss'])


ResNet18(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [121]:
print(f"VGG16 -> Model2 Keys")
print(fetch_model_vgg16_D3.keys())
model2.load_state_dict(fetch_model_vgg16_D3['state_dict'])
model2.eval()

VGG16 -> Model2 Keys
dict_keys(['epoch', 'state_dict', 'optimizer', 'train_loss', 'val_loss'])


VGG16(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

In [122]:
print(f"MOBILENETV2 -> Model3 Keys")
print(fetch_model_MV2_D3.keys())
model3.load_state_dict(fetch_model_MV2_D3['state_dict'])
model3.eval()

MOBILENETV2 -> Model3 Keys
dict_keys(['epoch', 'state_dict', 'optimizer', 'train_loss', 'val_loss'])


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [123]:
criterion1 = nn.CrossEntropyLoss()

model1.eval()
with torch.no_grad():
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(test_dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model1(inputs)
        loss = criterion1(outputs, labels)
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Calculate the test loss and accuracy
    test_loss = running_loss / len(test_dataloader)
    test_accuracy = 100 * correct / total
    print('Test Loss: %.4f, Test Accuracy: %.2f %%' % (test_loss, test_accuracy))

Test Loss: 0.3366, Test Accuracy: 89.42 %


In [124]:
criterion2 = nn.CrossEntropyLoss()

model2.eval()
with torch.no_grad():
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(datasetLoader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model2(inputs)
        loss = criterion2(outputs, labels)
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    test_loss = running_loss / len(datasetLoader)
    test_accuracy = 100 * correct / total
    print('Test Loss: %.4f, Test Accuracy: %.2f %%' % (test_loss, test_accuracy))

Test Loss: 0.7392, Test Accuracy: 75.14 %


In [125]:
criterion3 = nn.CrossEntropyLoss()

model3.eval()
with torch.no_grad():
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(datasetLoader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model3(inputs)
        loss = criterion3(outputs, labels)
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    test_loss = running_loss / len(datasetLoader)
    test_accuracy = 100 * correct / total
    print('Test Loss: %.4f, Test Accuracy: %.2f %%' % (test_loss, test_accuracy))

Test Loss: 0.4546, Test Accuracy: 84.71 %
